# Unit 2: Linear Equations and Elimination

In this unit:
1. Linear equations
1. Gaussian elimination
1. Pivots
1. Rank
1. LU factorization
1. Taylor series expansions with Jacobians

### Linear Equations

$$
\begin{array}{rcl}
x - 2y &=& 1\\
3x + 2y &=& 11
\end{array}
$$

or
$$
\begin{bmatrix}
1 & -2 \\
3 & 2
\end{bmatrix}
\begin{bmatrix}
x \\
y
\end{bmatrix}
=
\begin{bmatrix}
1 \\
11
\end{bmatrix}
$$

Multiply first equation by $3$ and subtract from second equation:

$$
\begin{array}{rcl}
x - 2y &=& 1\\
 8y &=& 8
\end{array}
$$

or
$$
\begin{bmatrix}
1 & -2 \\
0 & 8
\end{bmatrix}
\begin{bmatrix}
x \\
y
\end{bmatrix}
=
\begin{bmatrix}
1 \\
8
\end{bmatrix}
$$

Now $y=1$ and from this we can find $x$

$1x -2 \times 1 = 1$

So $x = 3$

So solution is $[3,1]^T$

In [1]:
A = [1 -2; 3 2]; b=[1,11];

In [2]:
A*[3,1] == b, A*[3,1] - b

(true, [0, 0])

In [3]:
A2 = copy(A) #doing the row operation manually
A2[2,:] = A[2,:] - 3A[1,:];
A2

2×2 Array{Int64,2}:
 1  -2
 0   8

Now y = 1

First equations becomes: $1x - 2\times1 = 1$

So $x = 3$

In [4]:
A*[3,1]

2-element Array{Int64,1}:
  1
 11

In [5]:
A*[3,1] == [1,11]

true

In [6]:
b = [1,11]
A \ b #solving using the `backslash' operator

2-element Array{Float64,1}:
 3.0
 1.0

In [7]:
using LinearAlgebra
A = rand(1000,1000)
b = rand(1000);
xSol = A \ b
norm(A*xSol - b)

1.2581267071498832e-12

In [8]:
? \

search: \



```
\(x, y)
```

Left division operator: multiplication of `y` by the inverse of `x` on the left. Gives floating-point results for integer arguments.

# Examples

```jldoctest
julia> 3 \ 6
2.0

julia> inv(3) * 6
2.0

julia> A = [4 3; 2 1]; x = [5, 6];

julia> A \ x
2-element Array{Float64,1}:
  6.5
 -7.0

julia> inv(A) * x
2-element Array{Float64,1}:
  6.5
 -7.0
```

---

```
\(A, B)
```

Matrix division using a polyalgorithm. For input matrices `A` and `B`, the result `X` is such that `A*X == B` when `A` is square. The solver that is used depends upon the structure of `A`.  If `A` is upper or lower triangular (or diagonal), no factorization of `A` is required and the system is solved with either forward or backward substitution. For non-triangular square matrices, an LU factorization is used.

For rectangular `A` the result is the minimum-norm least squares solution computed by a pivoted QR factorization of `A` and a rank estimate of `A` based on the R factor.

When `A` is sparse, a similar polyalgorithm is used. For indefinite matrices, the `LDLt` factorization does not use pivoting during the numerical factorization and therefore the procedure can fail even for invertible matrices.

# Examples

```jldoctest
julia> A = [1 0; 1 -2]; B = [32; -4];

julia> X = A \ B
2-element Array{Float64,1}:
 32.0
 18.0

julia> A * X == B
true
```

---

```
(\)(F::QRSparse, B::StridedVecOrMat)
```

Solve the least squares problem $\min\|Ax - b\|^2$ or the linear system of equations $Ax=b$ when `F` is the sparse QR factorization of $A$. A basic solution is returned when the problem is underdetermined.

# Examples

```jldoctest
julia> A = sparse([1,2,4], [1,1,1], [1.0,1.0,1.0], 4, 2)
4×2 SparseMatrixCSC{Float64,Int64} with 3 stored entries:
  [1, 1]  =  1.0
  [2, 1]  =  1.0
  [4, 1]  =  1.0

julia> qr(A)\fill(1.0, 4)
2-element Array{Float64,1}:
 1.0
 0.0
```


$$
A x = b
$$

Left mulitply $A^{-1}$ to $I x = A^{-1} b$ or $x = A^{-1} b$

In [9]:
A = [1 -2; 3 2]; b=[1,11];
inv(A)*b

2-element Array{Float64,1}:
 3.0
 1.0

$$
\begin{array}{rcl}
x - 2y &=& 1\\
2x -4 y &=& 11
\end{array}
$$

In [10]:
det([1 -2; 
     2 -4])

0.0

$$
\begin{array}{rcl}
x - 2y &=& 1\\
3x -4 y &=& 11 \\
7x -2y &=& 12
\end{array}
$$

Factorizations: $A = U^T \Sigma V$,  $A = QR $ or today...

# A bigger system towards $PA = LU$  or sometimes $A = LU$

In [1]:
A = [ 10.0   2.0  2.0  3.0;
      3.0  12.0  8.0  1.0;
      2.0   4.0  8.0  8.0;
      1.0   4.0  5.0  0.0]

#You can think of Ax = b

4×4 Array{Float64,2}:
 10.0   2.0  2.0  3.0
  3.0  12.0  8.0  1.0
  2.0   4.0  8.0  8.0
  1.0   4.0  5.0  0.0

In [2]:
ℓ21 = 3/10; ℓ31 = 2/10; ℓ41 = 1/10;   #ℓ \ell + [TAB]

In [3]:
#after Gaussian Elimination stpes on first col
A1 = [
        A[1,:]'   #first row unchanges
        (A[2,:] - ℓ21*A[1,:])'
        (A[3,:] - ℓ31*A[1,:])'
        (A[4,:] - ℓ41*A[1,:])'
]

4×4 Array{Float64,2}:
 10.0   2.0  2.0   3.0
  0.0  11.4  7.4   0.1
  0.0   3.6  7.6   7.4
  0.0   3.8  4.8  -0.3

In [4]:
E21 = [1 0 0 0 ;
       -ℓ21 1 0 0 
       0   0  1 0;
       0   0  0 1] #Elementary matrix

#Second row of the matrix product E21*A is -ℓ21 * first row of A + 1 * second row of A + zeros

4×4 Array{Float64,2}:
  1.0  0.0  0.0  0.0
 -0.3  1.0  0.0  0.0
  0.0  0.0  1.0  0.0
  0.0  0.0  0.0  1.0

In [5]:
E21*A

4×4 Array{Float64,2}:
 10.0   2.0  2.0  3.0
  0.0  11.4  7.4  0.1
  2.0   4.0  8.0  8.0
  1.0   4.0  5.0  0.0

In [6]:
inv(E21)

4×4 Array{Float64,2}:
 1.0  0.0  0.0  0.0
 0.3  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0

In [7]:
E31 = [1 0 0 0 ;
       0 1 0 0 
       -ℓ31   0  1 0;
       0   0  0 1]

4×4 Array{Float64,2}:
  1.0  0.0  0.0  0.0
  0.0  1.0  0.0  0.0
 -0.2  0.0  1.0  0.0
  0.0  0.0  0.0  1.0

In [8]:
E41 = [1 0 0 0;
       0 1 0 0;
        0 0 1 0
        -ℓ41 0 0 1]

4×4 Array{Float64,2}:
  1.0  0.0  0.0  0.0
  0.0  1.0  0.0  0.0
  0.0  0.0  1.0  0.0
 -0.1  0.0  0.0  1.0

In [9]:
E41*E31*E21 # *A

4×4 Array{Float64,2}:
  1.0  0.0  0.0  0.0
 -0.3  1.0  0.0  0.0
 -0.2  0.0  1.0  0.0
 -0.1  0.0  0.0  1.0

In [10]:
E31*E21*E41 #they commute

4×4 Array{Float64,2}:
  1.0  0.0  0.0  0.0
 -0.3  1.0  0.0  0.0
 -0.2  0.0  1.0  0.0
 -0.1  0.0  0.0  1.0

In [11]:
ℓ1 = E41*E31*E21

4×4 Array{Float64,2}:
  1.0  0.0  0.0  0.0
 -0.3  1.0  0.0  0.0
 -0.2  0.0  1.0  0.0
 -0.1  0.0  0.0  1.0

In [12]:
ℓ1*A

4×4 Array{Float64,2}:
 10.0   2.0  2.0   3.0
  0.0  11.4  7.4   0.1
  0.0   3.6  7.6   7.4
  0.0   3.8  4.8  -0.3

In [13]:
A1

4×4 Array{Float64,2}:
 10.0   2.0  2.0   3.0
  0.0  11.4  7.4   0.1
  0.0   3.6  7.6   7.4
  0.0   3.8  4.8  -0.3

In [14]:
ℓ32 = 3.6/11.4; ℓ42 = 3.8/11.4;

In [15]:
A2 = [
    A1[1,:]'; #which is also first row of A
    A1[2,:]';
    (A1[3,:] - ℓ32*A1[2,:])';
    (A1[4,:] - ℓ42*A1[2,:])'
]

4×4 Array{Float64,2}:
 10.0   2.0  2.0       3.0
  0.0  11.4  7.4       0.1
  0.0   0.0  5.26316   7.36842
  0.0   0.0  2.33333  -0.333333

In [17]:
ℓ2 = [1 0 0 0;
      0 1 0 0;
      0 -ℓ32 1 0;
      0 -ℓ42 0 1]

4×4 Array{Float64,2}:
 1.0   0.0       0.0  0.0
 0.0   1.0       0.0  0.0
 0.0  -0.315789  1.0  0.0
 0.0  -0.333333  0.0  1.0

In [18]:
ℓ2*(ℓ1*A)

4×4 Array{Float64,2}:
 10.0   2.0  2.0       3.0
  0.0  11.4  7.4       0.1
  0.0   0.0  5.26316   7.36842
  0.0   0.0  2.33333  -0.333333

In [19]:
A2

4×4 Array{Float64,2}:
 10.0   2.0  2.0       3.0
  0.0  11.4  7.4       0.1
  0.0   0.0  5.26316   7.36842
  0.0   0.0  2.33333  -0.333333

In [24]:
ℓ43 = 2.3333333333333333333333333333333333333/5.26316

0.44333315600007095

In [25]:
A3 = [
    A2[1,:]';
    A2[2,:]';
    A2[3,:]';
    (A2[4,:] - ℓ43*A2[3,:])'
]

4×4 Array{Float64,2}:
 10.0   2.0  2.0          3.0
  0.0  11.4  7.4          0.1
  0.0   0.0  5.26316      7.36842
  0.0   0.0  9.33333e-7  -3.6

In [26]:
ℓ3 =[1 0 0 0 ;
     0 1 0 0 ;
     0 0 1 0 ;
     0 0  -ℓ43 1]

4×4 Array{Float64,2}:
 1.0  0.0   0.0       0.0
 0.0  1.0   0.0       0.0
 0.0  0.0   1.0       0.0
 0.0  0.0  -0.443333  1.0

In [27]:
ℓ3*(ℓ2*(ℓ1*A))

4×4 Array{Float64,2}:
 10.0   2.0  2.0          3.0
  0.0  11.4  7.4          0.1
  0.0   0.0  5.26316      7.36842
  0.0   0.0  9.33333e-7  -3.6

In [30]:
using LinearAlgebra
F = lu(A)

LU{Float64,Array{Float64,2}}
L factor:
4×4 Array{Float64,2}:
 1.0  0.0       0.0       0.0
 0.3  1.0       0.0       0.0
 0.2  0.315789  1.0       0.0
 0.1  0.333333  0.443333  1.0
U factor:
4×4 Array{Float64,2}:
 10.0   2.0  2.0       3.0
  0.0  11.4  7.4       0.1
  0.0   0.0  5.26316   7.36842
  0.0   0.0  0.0      -3.6

In [32]:
? lu

search: lu lu! LU flush values include include_string include_dependency lmul!



```
lu(A, pivot=Val(true); check = true) -> F::LU
```

Compute the LU factorization of `A`.

When `check = true`, an error is thrown if the decomposition fails. When `check = false`, responsibility for checking the decomposition's validity (via [`issuccess`](@ref)) lies with the user.

In most cases, if `A` is a subtype `S` of `AbstractMatrix{T}` with an element type `T` supporting `+`, `-`, `*` and `/`, the return type is `LU{T,S{T}}`. If pivoting is chosen (default) the element type should also support `abs` and `<`.

The individual components of the factorization `F` can be accessed via `getproperty`:

| Component | Description                         |
|:--------- |:----------------------------------- |
| `F.L`     | `L` (lower triangular) part of `LU` |
| `F.U`     | `U` (upper triangular) part of `LU` |
| `F.p`     | (right) permutation `Vector`        |
| `F.P`     | (right) permutation `Matrix`        |

Iterating the factorization produces the components `F.L`, `F.U`, and `F.p`.

The relationship between `F` and `A` is

`F.L*F.U == A[F.p, :]`

`F` further supports the following functions:

| Supported function  | `LU` | `LU{T,Tridiagonal{T}}` |
|:------------------- |:---- |:---------------------- |
| [`/`](@ref)         | ✓    |                        |
| [`\`](@ref)         | ✓    | ✓                      |
| [`inv`](@ref)       | ✓    | ✓                      |
| [`det`](@ref)       | ✓    | ✓                      |
| [`logdet`](@ref)    | ✓    | ✓                      |
| [`logabsdet`](@ref) | ✓    | ✓                      |
| [`size`](@ref)      | ✓    | ✓                      |

# Examples

```jldoctest
julia> A = [4 3; 6 3]
2×2 Array{Int64,2}:
 4  3
 6  3

julia> F = lu(A)
LU{Float64,Array{Float64,2}}
L factor:
2×2 Array{Float64,2}:
 1.0       0.0
 0.666667  1.0
U factor:
2×2 Array{Float64,2}:
 6.0  3.0
 0.0  1.0

julia> F.L * F.U == A[F.p, :]
true

julia> l, u, p = lu(A); # destructuring via iteration

julia> l == F.L && u == F.U && p == F.p
true
```

---

```
lu(A::SparseMatrixCSC; check = true) -> F::UmfpackLU
```

Compute the LU factorization of a sparse matrix `A`.

For sparse `A` with real or complex element type, the return type of `F` is `UmfpackLU{Tv, Ti}`, with `Tv` = [`Float64`](@ref) or `ComplexF64` respectively and `Ti` is an integer type ([`Int32`](@ref) or [`Int64`](@ref)).

When `check = true`, an error is thrown if the decomposition fails. When `check = false`, responsibility for checking the decomposition's validity (via [`issuccess`](@ref)) lies with the user.

The individual components of the factorization `F` can be accessed by indexing:

| Component | Description                         |
|:--------- |:----------------------------------- |
| `L`       | `L` (lower triangular) part of `LU` |
| `U`       | `U` (upper triangular) part of `LU` |
| `p`       | right permutation `Vector`          |
| `q`       | left permutation `Vector`           |
| `Rs`      | `Vector` of scaling factors         |
| `:`       | `(L,U,p,q,Rs)` components           |

The relation between `F` and `A` is

`F.L*F.U == (F.Rs .* A)[F.p, F.q]`

`F` further supports the following functions:

  * [`\`](@ref)
  * [`cond`](@ref)
  * [`det`](@ref)

!!! note
    `lu(A::SparseMatrixCSC)` uses the UMFPACK library that is part of SuiteSparse. As this library only supports sparse matrices with [`Float64`](@ref) or `ComplexF64` elements, `lu` converts `A` into a copy that is of type `SparseMatrixCSC{Float64}` or `SparseMatrixCSC{ComplexF64}` as appropriate.



In [31]:
typeof(F)

LU{Float64,Array{Float64,2}}

In [33]:
L, U = lu(A)

LU{Float64,Array{Float64,2}}
L factor:
4×4 Array{Float64,2}:
 1.0  0.0       0.0       0.0
 0.3  1.0       0.0       0.0
 0.2  0.315789  1.0       0.0
 0.1  0.333333  0.443333  1.0
U factor:
4×4 Array{Float64,2}:
 10.0   2.0  2.0       3.0
  0.0  11.4  7.4       0.1
  0.0   0.0  5.26316   7.36842
  0.0   0.0  0.0      -3.6

In [34]:
L

4×4 Array{Float64,2}:
 1.0  0.0       0.0       0.0
 0.3  1.0       0.0       0.0
 0.2  0.315789  1.0       0.0
 0.1  0.333333  0.443333  1.0

In [35]:
[1 0 0 0;
 ℓ21 1 0 0;
 ℓ31 ℓ32 1 0
 ℓ41 ℓ42 ℓ43 1] #L manually done

4×4 Array{Float64,2}:
 1.0  0.0       0.0       0.0
 0.3  1.0       0.0       0.0
 0.2  0.315789  1.0       0.0
 0.1  0.333333  0.443333  1.0

In [36]:
U

4×4 Array{Float64,2}:
 10.0   2.0  2.0       3.0
  0.0  11.4  7.4       0.1
  0.0   0.0  5.26316   7.36842
  0.0   0.0  0.0      -3.6

In [37]:
A3

4×4 Array{Float64,2}:
 10.0   2.0  2.0          3.0
  0.0  11.4  7.4          0.1
  0.0   0.0  5.26316      7.36842
  0.0   0.0  9.33333e-7  -3.6

In [38]:
L*U

4×4 Array{Float64,2}:
 10.0   2.0  2.0   3.0
  3.0  12.0  8.0   1.0
  2.0   4.0  8.0   8.0
  1.0   4.0  5.0  -4.44089e-16

In [39]:
A

4×4 Array{Float64,2}:
 10.0   2.0  2.0  3.0
  3.0  12.0  8.0  1.0
  2.0   4.0  8.0  8.0
  1.0   4.0  5.0  0.0

In [31]:
inv(ℓ3*ℓ2*ℓ1)

4×4 Array{Float64,2}:
 1.0  0.0       0.0       0.0
 0.3  1.0       0.0       0.0
 0.2  0.315789  1.0       0.0
 0.1  0.333333  0.443333  1.0

In [40]:
L, U, p = lu(A);
display(L)
display(U)
display(p)

4×4 Array{Float64,2}:
 1.0  0.0       0.0       0.0
 0.3  1.0       0.0       0.0
 0.2  0.315789  1.0       0.0
 0.1  0.333333  0.443333  1.0

4×4 Array{Float64,2}:
 10.0   2.0  2.0       3.0
  0.0  11.4  7.4       0.1
  0.0   0.0  5.26316   7.36842
  0.0   0.0  0.0      -3.6

4-element Array{Int64,1}:
 1
 2
 3
 4

In [42]:
F = lu(A)
F.P

4×4 Array{Float64,2}:
 1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0

In [43]:
A-L*U

4×4 Array{Float64,2}:
  0.0          0.0  0.0          0.0
 -4.44089e-16  0.0  0.0          0.0
  0.0          0.0  0.0          0.0
  0.0          0.0  8.88178e-16  4.44089e-16

In [44]:
As = [ 
    3.0  12.0  8.0  1.0;
    10.0   2.0  2.0  3.0;  #Aswitched  (switched second and first row)  
      2.0   4.0  8.0  8.0;
      1.0   4.0  5.0  0.0]

4×4 Array{Float64,2}:
  3.0  12.0  8.0  1.0
 10.0   2.0  2.0  3.0
  2.0   4.0  8.0  8.0
  1.0   4.0  5.0  0.0

In [48]:
Ls, Us, ps = lu(As)

LU{Float64,Array{Float64,2}}
L factor:
4×4 Array{Float64,2}:
 1.0  0.0       0.0       0.0
 0.3  1.0       0.0       0.0
 0.2  0.315789  1.0       0.0
 0.1  0.333333  0.443333  1.0
U factor:
4×4 Array{Float64,2}:
 10.0   2.0  2.0       3.0
  0.0  11.4  7.4       0.1
  0.0   0.0  5.26316   7.36842
  0.0   0.0  0.0      -3.6

In [49]:
Ls*Us - As  #bad... A does not equal L*U in this case

4×4 Array{Float64,2}:
  7.0  -10.0  -6.0           2.0
 -7.0   10.0   6.0          -2.0
  0.0    0.0   0.0           0.0
  0.0    0.0  -8.88178e-16  -4.44089e-16

In [51]:
ps

4-element Array{Int64,1}:
 2
 1
 3
 4

In [52]:
L*U - As[ps,:]

4×4 Array{Float64,2}:
 0.0          0.0   0.0           0.0
 4.44089e-16  0.0   0.0           0.0
 0.0          0.0   0.0           0.0
 0.0          0.0  -8.88178e-16  -4.44089e-16

In [56]:
F = lu(As)

LU{Float64,Array{Float64,2}}
L factor:
4×4 Array{Float64,2}:
 1.0  0.0       0.0       0.0
 0.3  1.0       0.0       0.0
 0.2  0.315789  1.0       0.0
 0.1  0.333333  0.443333  1.0
U factor:
4×4 Array{Float64,2}:
 10.0   2.0  2.0       3.0
  0.0  11.4  7.4       0.1
  0.0   0.0  5.26316   7.36842
  0.0   0.0  0.0      -3.6

In [57]:
F.L*F.U - F.P*As     #capital P is the permuation matrix

4×4 Array{Float64,2}:
 0.0          0.0   0.0           0.0
 4.44089e-16  0.0   0.0           0.0
 0.0          0.0   0.0           0.0
 0.0          0.0  -8.88178e-16  -4.44089e-16

# A permutation matrix is like...

In [53]:
PP = [0 1 0 0;
      1 0 0 0;
      0 0 1 0;
      0 0 0 1]
PP*A

4×4 Array{Float64,2}:
  3.0  12.0  8.0  1.0
 10.0   2.0  2.0  3.0
  2.0   4.0  8.0  8.0
  1.0   4.0  5.0  0.0

In [54]:
As

4×4 Array{Float64,2}:
  3.0  12.0  8.0  1.0
 10.0   2.0  2.0  3.0
  2.0   4.0  8.0  8.0
  1.0   4.0  5.0  0.0

In [55]:
PP*PP

4×4 Array{Int64,2}:
 1  0  0  0
 0  1  0  0
 0  0  1  0
 0  0  0  1

#How to compute det using $A=LU$

In [58]:
A

4×4 Array{Float64,2}:
 10.0   2.0  2.0  3.0
  3.0  12.0  8.0  1.0
  2.0   4.0  8.0  8.0
  1.0   4.0  5.0  0.0

In [59]:
L,U = lu(A)

LU{Float64,Array{Float64,2}}
L factor:
4×4 Array{Float64,2}:
 1.0  0.0       0.0       0.0
 0.3  1.0       0.0       0.0
 0.2  0.315789  1.0       0.0
 0.1  0.333333  0.443333  1.0
U factor:
4×4 Array{Float64,2}:
 10.0   2.0  2.0       3.0
  0.0  11.4  7.4       0.1
  0.0   0.0  5.26316   7.36842
  0.0   0.0  0.0      -3.6

In [61]:
diag(U) #pivots

4-element Array{Float64,1}:
 10.0
 11.4
  5.2631578947368425
 -3.599999999999999

In [66]:
pivots = diag(U)
prod = 1
for i in 1:length(pivots)
    prod *= pivots[i]
end
prod

-2159.9999999999995

In [67]:
det(A)

-2159.9999999999995

# But what if $P \neq I$? 

$$
A x = b
$$

$$
LU x = b
$$

$$ L(Ux) = b$$ 

or with $y=Ux$,

$$ Ly = b$$ 

So first solve for $y$ and then solve,
$$
Ux = y
$$

Can also use for finding inverses, and determinents.

Both LU and QR decompoistions are of the form A = WU where U is upper triangular 

# Back substitution 
(See Alg. 11.1, pg207 in [VMLS])

In [28]:
#Implement it!

In [104]:
function solveUpperTri(U,b)
    n = size(U)[1] 
    x = zeros(n)
    for i in n:-1:1
#         println("Hey man I am here")
        prod = dot(x[i+1:n],U[i,i+1:n])
#         @show prod
        x[i] = (b[i] - prod)/U[i,i]
    end
    x
end

solveUpperTri (generic function with 1 method)

In [69]:
U = [1 -2;
     0 8]
b= [1,8]

2-element Array{Int64,1}:
 1
 8

In [77]:
solveUpperTri(U,b)

Hey man I am here
prod = 0.0
Hey man I am here
prod = -2.0


2-element Array{Float64,1}:
 3.0
 1.0

In [100]:
function solveLowerTri(L,b)
    n = size(L)[1] 
    x = zeros(n)
    for i in 1:n
        prod = dot(x[1:i-1],L[i,1:i-1])
        x[i] = (b[i] - prod)/L[i,i]
    end
    x
end

solveLowerTri (generic function with 1 method)

In [98]:
methods(solveLowerTri)

# 1 method for generic function "solveLowerTri":
[1] solveLowerTri(L, b) in Main at In[96]:2

In [81]:
LL = [2 0 0;
      3 2 0;
      -1 1 3]
b = [1,1,2]

x = LL \ b

3-element Array{Float64,1}:
  0.5
 -0.25
  0.9166666666666666

In [82]:
LL*x - b

3-element Array{Float64,1}:
 0.0
 0.0
 0.0

In [87]:
solveLowerTri(LL,b)

3-element Array{Float64,1}:
  0.5
 -0.25
  0.9166666666666666

In [102]:
function usingLUtoSolveIt(A,b)
   L, U = lu(A) #Julia lu(decomposion)
   y = solveLowerTri(L,b)
   return solveUpperTri(U,y)
end

usingLUtoSolveIt (generic function with 1 method)

In [91]:
A
b=ones(4);

In [92]:
xSol = A \ b

4-element Array{Float64,1}:
  0.10277777777777779
 -0.11250000000000003
  0.2694444444444445
 -0.11388888888888896

In [93]:
norm(A*xSol - b)

3.1401849173675503e-16

In [105]:
usingLUtoSolveIt(A,b)

4-element Array{Float64,1}:
  0.10277777777777782
 -0.11250000000000003
  0.2694444444444445
 -0.11388888888888896

$$
A = LU 
$$

$$
A = QR
$$

In [3]:
#using Pkg
#Pkg.add("RowEchelon")

   Updating registry at `~/.juliapro/JuliaPro_v1.4.2-1/registries/JuliaPro`
  Resolving package versions...
   Updating `~/Project.toml`
 [no changes]
   Updating `~/Manifest.toml`
 [no changes]


In [19]:
using RowEchelon

In [5]:
? rref

search: rref rref! rref_with_pivots rref_with_pivots! searchsortedfirst



```
rref(A)
```

Compute the reduced row echelon form of the matrix A. Since this algorithm is sensitive to numerical imprecision,

  * Complex numbers are converted to ComplexF64
  * Integer, Float16 and Float32 numbers are converted to Float64
  * Rational are kept unchanged

```jldoctest
julia> rref([ 1  2 -1  -4;
              2  3 -1 -11;
             -2  0 -3  22])
3×4 Array{Float64,2}:
 1.0  0.0  0.0  -8.0
 0.0  1.0  0.0   1.0
 0.0  0.0  1.0  -2.0

julia> rref([16  2  3  13;
              5 11 10   8;
              9  7  6  12;
              4 14 15   1])
4×4 Array{Float64,2}:
 1.0  0.0  0.0   1.0
 0.0  1.0  0.0   3.0
 0.0  0.0  1.0  -3.0
 0.0  0.0  0.0   0.0

julia> rref([ 1  2  0   3;
              2  4  0   7])
2×4 Array{Float64,2}:
 1.0  2.0  0.0  0.0
 0.0  0.0  0.0  1.0
```


In [21]:
rref([A [1,11]])

2×3 Array{Float64,2}:
 1.0  0.0  3.0
 0.0  1.0  1.0

### Computing the inverse

In [7]:
using LinearAlgebra
[A I]

2×4 Array{Int64,2}:
 1  -2  1  0
 3   2  0  1

In [8]:
R = rref([A I])

2×4 Array{Float64,2}:
 1.0  0.0   0.25   0.25
 0.0  1.0  -0.375  0.125

In [66]:
inv(A)

2×2 Array{Float64,2}:
  0.25   0.25
 -0.375  0.125

In [10]:
R[:,3:4]

2×2 Array{Float64,2}:
  0.25   0.25
 -0.375  0.125

### Rank

In [22]:
A = [1 2 3 4;
     5 6 7 8;
     2 4 6 8]

3×4 Array{Int64,2}:
 1  2  3  4
 5  6  7  8
 2  4  6  8

In [23]:
rref(A)

3×4 Array{Float64,2}:
 1.0  0.0  -1.0  -2.0
 0.0  1.0   2.0   3.0
 0.0  0.0   0.0   0.0

In [24]:
rank(A)

2

In [31]:
B = [1 2 3 4;
     5 6 7 8;
     2 4 6 9]

3×4 Array{Int64,2}:
 1  2  3  4
 5  6  7  8
 2  4  6  9

In [32]:
rank(B)

3

In [33]:
rref(B)

3×4 Array{Float64,2}:
 1.0  0.0  -1.0  0.0
 0.0  1.0   2.0  0.0
 0.0  0.0   0.0  1.0

In [46]:
A = [1 2 3;
     4 5 6;
     2 4 6]

3×3 Array{Int64,2}:
 1  2  3
 4  5  6
 2  4  6

In [47]:
rank(A)

2

In [48]:
det(A)

0.0

In [49]:
inv(A)

SingularException: SingularException(3)

In [43]:
A = [1,10,2]*[1,2,3]'

3×3 Array{Int64,2}:
  1   2   3
 10  20  30
  2   4   6

In [44]:
rank(A)

1

In [45]:
inv(A)

SingularException: SingularException(2)

### Taylor Series
See [VMLS] Appendix C

$$
f: \mathbb{R}^{n} \rightarrow \mathbb{R}
$$

$$
\nabla f(z)=\left[\begin{array}{c}\frac{\partial f}{\partial x_{1}}(z) \\ \vdots \\ \frac{\partial f}{\partial x_{n}}(z)\end{array}\right]
$$

$$
\hat{f}(x ; z)=f(z)+\frac{\partial f}{\partial x_{1}}(z)\left(x_{1}-z_{1}\right)+\cdots+\frac{\partial f}{\partial x_{n}}(z)\left(x_{n}-z_{n}\right)
$$

$$
\hat{f}(x ; z)=f(z)+\nabla f(z)^{T}(x-z)
$$

$$
f: \mathbb{R}^{n} \rightarrow \mathbb{R}^{m}
$$

$$
f(x)=\left[\begin{array}{c}f_{1}(x) \\ \vdots \\ f_{m}(x)\end{array}\right]
$$

Jacobian:
$$
D f(z)_{i j}=\frac{\partial f_{i}}{\partial x_{j}}(z), \quad i=1, \ldots, m, \quad j=1, \ldots, n
$$

rows are: $$\nabla f_{i}(z)^{T}, \text { for } i=1, \ldots, m$$

Taylor approximation:

$$
\begin{aligned} \hat{f}(x ; z)_{i} &=f_{i}(z)+\frac{\partial f_{i}}{\partial x_{1}}(z)\left(x_{1}-z_{1}\right)+\cdots+\frac{\partial f_{i}}{\partial x_{n}}(z)\left(x_{n}-z_{n}\right) \\ &=f_{i}(z)+\nabla f_{i}(z)^{T}(x-z) \end{aligned}
$$

$$
\hat{f}(x ; z)=f(z)+D f(z)(x-z)
$$